<a href="https://colab.research.google.com/github/juyounLee/DeepLearningZeroToAll_TF2./blob/main/05_01_Logistic_Regression_Binary.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import numpy as np
import math

In [2]:
# data
x_train = [[1., 2.],
          [2., 3.],
          [3., 1.],
          [4., 3.],
          [5., 3.],
          [6., 2.]]
y_train = [[0.],
          [0.],
          [0.],
          [1.],
          [1.],
          [1.]]

In [3]:
# 변수 W와 b를 초기화합니다.
W = tf.Variable(tf.random.normal([2,1]))
b = tf.Variable(tf.random.normal([1]))

print(W.numpy())
print(b.numpy())

[[-2.3354757 ]
 [ 0.18048511]]
[-0.4744055]


### tf.data, tf.data.Dataset

- 가지고 있는 데이터를 model에 넣기위해 적합한 텐서로 변환한다.

- tf.data.Dataset 은 데이터의 병렬 처리를 쉽게 하기 위하여 GPU가 연산이 끝나면 다음 데이터를 즉시 Pre-Fetch 처리하기 위한 목적이다. 

- tf.data.Dataset.from_tensor_slices(): 가지고 있는 데이터 source를 여러개의 Tensor로 분할한다.

In [4]:
dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(len(x_train))

In [5]:
# 선형회귀와 같은 형태로 예측한다.

def hypo_func(feature):
  hypothesis = tf.matmul(feature, W) + b
  return hypothesis

In [6]:
# Classification을 위해 hypothesis값을 'sigmoid'함수를 이용하여 0에서 1 사이의 값으로 변환시켜준다. 
def sig_func(feature):
  h = hypo_func(feature)
  sigmoid = tf.sigmoid(h)
  return sigmoid

In [7]:
# sigmoid output을 위한 cost function. 
def cost_func(feature, label):
  s = sig_func(feature)
  cost = -tf.reduce_mean(label*tf.math.log(s) + (1-label)*tf.math.log(1-s))
  return cost

In [8]:
def grad(sigmoid, feature, label):
  with tf.GradientTape() as tape:
    loss_value = cost_func(feature, label)
  return tape.gradient(loss_value, [W,b])

In [9]:
# 변수들의 gradient 계산
optimizer = tf.keras.optimizers.SGD(learning_rate=0.001)

In [10]:
for step in range(1001):
  for feature, label in iter(dataset):
    grads = grad(sig_func(feature), feature, label)
    optimizer.apply_gradients(grads_and_vars=zip(grads,[W,b]))
    cost = cost_func(feature, label)

    if step%100 ==0:
      print('step: {} cost: {:.5f}'.format(step, cost))
    

step: 0 cost: 5.84273
step: 100 cost: 5.03990
step: 200 cost: 4.25673
step: 300 cost: 3.50774
step: 400 cost: 2.81477
step: 500 cost: 2.20626
step: 600 cost: 1.71498
step: 700 cost: 1.36924
step: 800 cost: 1.16259
step: 900 cost: 1.05022
step: 1000 cost: 0.98733


### 예측하기

In [11]:
# 예측할 data

x_test = [[5.,2.]]
y_test = [[1.]]

In [12]:
# sigmoid값이 0.5 이하이면 0의값을 그렇지 않으면 1의 값으로 return
def y_pred_func(sigmoid):
  y_pred = tf.cast(sigmoid > 0.5, dtype=tf.float32)
  return y_pred

In [13]:
sigmoid = sig_func(x_test)
y_prediction = y_pred_func(sigmoid)
print('y: ', y_prediction)

y:  tf.Tensor([[0.]], shape=(1, 1), dtype=float32)
